<a href="https://colab.research.google.com/github/rhutikcodes/weather-prediction-ml-model-analysis/blob/main/Weather_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!pip install plotly

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
import warnings

warnings.filterwarnings('ignore')

# 1. Import Necessary Libraries


In [59]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 2. Problem Statement

In this notebook we will learn how to implement how to apply logistic regression in a real world data. We are using open kaglle data set named, "Rain in Australia". It is a 10 year historical rainfall data that comes in csv file format. After doing primary data exploration, we will try to predict next day rainfall for different regions in australia.


# 3. Loading Dataset
We will first load and import the dataset to work with

In [60]:
df_raw = pd.read_csv('/content/drive/MyDrive/Final Year Project/weatherAUS.csv')
df = df_raw

# 4. Exploratory Data Analysis with Visualisation
In this step, we would analyze the data set, learn the statistics and correlation between different features

In [61]:
df_raw

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,2017-06-20,Uluru,3.5,21.8,0.0,NaN,NaN,E,31.0,ESE,...,27.0,1024.7,1021.2,NaN,NaN,9.4,20.9,No,0.0,No
142189,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,0.0,No
142190,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,0.0,No
142191,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,0.0,No


We can see that this dataset contains over 142,193 rows and 24 columns including date, numeric and categorical columns. We will create a model which will predict the RainTomorrow by using other features.

In [62]:
df_raw.shape

(142193, 24)

In [63]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

There are significant amount of missing values in the dataset especially in columns like "sunshine". "Evaporation","Cloud3pm", "cloud9pm". It is also observed that there are different types of data type like object, float64, int64.

Lets look at the basic statistics of the data

> Indented block



In [64]:
df_raw.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RISK_MM
count,141556.000000,141871.000000,140787.000000,81350.000000,74377.000000,132923.000000,140845.000000,139563.000000,140419.000000,138583.000000,128179.000000,128212.000000,88536.000000,85099.000000,141289.000000,139467.000000,142193.000000
mean,12.186400,23.226784,2.349974,5.469824,7.624853,39.984292,14.001988,18.637576,68.843810,51.482606,1017.653758,1015.258204,4.437189,4.503167,16.987509,21.687235,2.360682
std,6.403283,7.117618,8.465173,4.188537,3.781525,13.588801,8.893337,8.803345,19.051293,20.797772,7.105476,7.036677,2.887016,2.720633,6.492838,6.937594,8.477969
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000,0.000000
25%,7.600000,17.900000,0.000000,2.600000,4.900000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.900000,1010.400000,1.000000,2.000000,12.300000,16.600000,0.000000
50%,12.000000,22.600000,0.000000,4.800000,8.500000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.600000,1015.200000,5.000000,5.000000,16.700000,21.100000,0.000000
75%,16.800000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.400000,1020.000000,7.000000,7.000000,21.600000,26.400000,0.800000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.700000,371.000000


In [65]:
#size of data... there are 3.345.580 of elements
df_raw.size

3412632

In [66]:
# Identification all column listed
df_raw.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')

**Identify Data Categorical values**

In [67]:
# Checking data categorical
print("location :", df_raw['Location'].unique())
print("WindGustDir :", df_raw['WindGustDir'].unique())
print("WindDir9am :", df_raw['WindDir9am'].unique())
print("WindDir3pm :", df_raw['WindDir3pm'].unique())
print("RainToday :", df_raw['RainToday'].unique())
print("RainTomorrow :", df_raw['RainTomorrow'].unique())

location : ['Albury' 'BadgerysCreek' 'Cobar' 'CoffsHarbour' 'Moree' 'Newcastle'
 'NorahHead' 'NorfolkIsland' 'Penrith' 'Richmond' 'Sydney' 'SydneyAirport'
 'WaggaWagga' 'Williamtown' 'Wollongong' 'Canberra' 'Tuggeranong'
 'MountGinini' 'Ballarat' 'Bendigo' 'Sale' 'MelbourneAirport' 'Melbourne'
 'Mildura' 'Nhil' 'Portland' 'Watsonia' 'Dartmoor' 'Brisbane' 'Cairns'
 'GoldCoast' 'Townsville' 'Adelaide' 'MountGambier' 'Nuriootpa' 'Woomera'
 'Albany' 'Witchcliffe' 'PearceRAAF' 'PerthAirport' 'Perth' 'SalmonGums'
 'Walpole' 'Hobart' 'Launceston' 'AliceSprings' 'Darwin' 'Katherine'
 'Uluru']
WindGustDir : ['W' 'WNW' 'WSW' 'NE' 'NNW' 'N' 'NNE' 'SW' 'ENE' 'SSE' 'S' 'NW' 'SE' 'ESE'
 nan 'E' 'SSW']
WindDir9am : ['W' 'NNW' 'SE' 'ENE' 'SW' 'SSE' 'S' 'NE' nan 'SSW' 'N' 'WSW' 'ESE' 'E'
 'NW' 'WNW' 'NNE']
WindDir3pm : ['WNW' 'WSW' 'E' 'NW' 'W' 'SSE' 'ESE' 'ENE' 'NNW' 'SSW' 'SW' 'SE' 'N' 'S'
 'NNE' nan 'NE']
RainToday : ['No' 'Yes' nan]
RainTomorrow : ['No' 'Yes']


In [68]:
# Checking the label counts data categorical
print("location :", df_raw['Location'].value_counts())
print("WindGustDir :", df_raw['WindGustDir'].value_counts())
print("WindDir9am :", df_raw['WindDir9am'].value_counts())
print("WindDir3pm :", df_raw['WindDir3pm'].value_counts())
print("RainToday :", df_raw['RainToday'].value_counts())
print("RainTomorrow :", df_raw['RainTomorrow'].value_counts())

location : Canberra            3418
Sydney              3337
Perth               3193
Darwin              3192
Hobart              3188
Brisbane            3161
Adelaide            3090
Bendigo             3034
Townsville          3033
AliceSprings        3031
MountGambier        3030
Launceston          3028
Ballarat            3028
Albany              3016
Albury              3011
PerthAirport        3009
MelbourneAirport    3009
Mildura             3007
SydneyAirport       3005
Nuriootpa           3002
Sale                3000
Watsonia            2999
Tuggeranong         2998
Portland            2996
Woomera             2990
Cairns              2988
Cobar               2988
Wollongong          2983
GoldCoast           2980
WaggaWagga          2976
Penrith             2964
NorfolkIsland       2964
SalmonGums          2955
Newcastle           2955
CoffsHarbour        2953
Witchcliffe         2952
Richmond            2951
Dartmoor            2943
NorahHead           2929
BadgerysCreek 

Dealing with invalid dataset

In [69]:
x=df.iloc[:,[1,2,3,4,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]].values
y=df.iloc[:,-1].values.reshape(-1,1) #slicing genrates new list
y.shape

(142193, 1)

In [70]:
x,y

(array([['Albury', 13.4, 22.9, ..., 16.9, 21.8, 'No'],
        ['Albury', 7.4, 25.1, ..., 17.2, 24.3, 'No'],
        ['Albury', 12.9, 25.7, ..., 21.0, 23.2, 'No'],
        ...,
        ['Uluru', 3.6, 25.3, ..., 10.9, 24.5, 'No'],
        ['Uluru', 5.4, 26.9, ..., 12.5, 26.1, 'No'],
        ['Uluru', 7.8, 27.0, ..., 15.1, 26.0, 'No']], dtype=object),
 array([['No'],
        ['No'],
        ['No'],
        ...,
        ['No'],
        ['No'],
        ['No']], dtype=object))

#### Dealing with invalid dataset

In [71]:
x,y

(array([['Albury', 13.4, 22.9, ..., 16.9, 21.8, 'No'],
        ['Albury', 7.4, 25.1, ..., 17.2, 24.3, 'No'],
        ['Albury', 12.9, 25.7, ..., 21.0, 23.2, 'No'],
        ...,
        ['Uluru', 3.6, 25.3, ..., 10.9, 24.5, 'No'],
        ['Uluru', 5.4, 26.9, ..., 12.5, 26.1, 'No'],
        ['Uluru', 7.8, 27.0, ..., 15.1, 26.0, 'No']], dtype=object),
 array([['No'],
        ['No'],
        ['No'],
        ...,
        ['No'],
        ['No'],
        ['No']], dtype=object))

In [72]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
x=imputer.fit_transform(x)
y=imputer.fit_transform(y)

In [73]:
x

array([['Albury', 13.4, 22.9, ..., 16.9, 21.8, 'No'],
       ['Albury', 7.4, 25.1, ..., 17.2, 24.3, 'No'],
       ['Albury', 12.9, 25.7, ..., 21.0, 23.2, 'No'],
       ...,
       ['Uluru', 3.6, 25.3, ..., 10.9, 24.5, 'No'],
       ['Uluru', 5.4, 26.9, ..., 12.5, 26.1, 'No'],
       ['Uluru', 7.8, 27.0, ..., 15.1, 26.0, 'No']], dtype=object)

In [74]:
y

array([['No'],
       ['No'],
       ['No'],
       ...,
       ['No'],
       ['No'],
       ['No']], dtype=object)

In [75]:
from sklearn.preprocessing import LabelEncoder
le1=LabelEncoder()
x[:,0]=le1.fit_transform(x[:,0])
le2=LabelEncoder()
x[:,4]=le2.fit_transform(x[:,4])
le3=LabelEncoder()
x[:,6]=le3.fit_transform(x[:,6])
le4=LabelEncoder()
x[:,7]=le4.fit_transform(x[:,7])
le5=LabelEncoder()
x[:,-1]=le5.fit_transform(x[:,-1])
le6=LabelEncoder()
y=le6.fit_transform(y)

In [76]:
x,y

(array([[2, 13.4, 22.9, ..., 16.9, 21.8, 0],
        [2, 7.4, 25.1, ..., 17.2, 24.3, 0],
        [2, 12.9, 25.7, ..., 21.0, 23.2, 0],
        ...,
        [41, 3.6, 25.3, ..., 10.9, 24.5, 0],
        [41, 5.4, 26.9, ..., 12.5, 26.1, 0],
        [41, 7.8, 27.0, ..., 15.1, 26.0, 0]], dtype=object),
 array([0, 0, 0, ..., 0, 0, 0]))

In [77]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

Splitting data

In [78]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [79]:
x_train,y_train,x_test,y_test

(array([[ 1.56343583,  0.26993011, -0.08711307, ...,  0.28003274,
         -0.18253881, -0.53296232],
        [-1.10558082,  1.59987639,  0.86912716, ...,  1.45429598,
          0.85024239,  1.87630525],
        [-1.45676722, -0.38721982,  1.82536739, ...,  1.77876346,
          1.94120845, -0.53296232],
        ...,
        [-1.66747906, -0.04299843, -0.79023088, ..., -0.38435305,
         -0.66256387,  1.87630525],
        [ 0.50987663, -0.70014836,  0.39100704, ..., -0.39980388,
          0.42840218, -0.53296232],
        [ 1.63367311,  1.22436215,  1.16443664, ...,  1.40794349,
          0.16657033, -0.53296232]]),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([[-1.10558082,  0.81755505,  0.81287773, ...,  1.40794349,
          0.89388103, -0.53296232],
        [-1.24605538, -0.8409662 , -1.24022629, ..., -1.01783717,
         -1.30259729, -0.53296232],
        [ 1.35272399, -0.32463412, -1.09960272, ..., -0.72427136,
         -1.25895865,  1.87630525],
        ...,
        [-0.4734453 ,

**Models**





1.Random Forest

In [80]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(n_estimators=200,random_state=0)


In [ ]:
classifier.fit(x_train,y_train)

In [ ]:
classifier.score(x_train,y_train)

In [ ]:
y_pred=classifier.predict(x_test)

In [ ]:
y_pred

In [ ]:
# y_pred=le6.inverse_transform(y_pred)

In [ ]:
# y_pred

In [ ]:
# y_test=le6.inverse_transform(y_test)

In [ ]:
# y_test

In [ ]:
# y_pred=y_pred.reshape(-1,1)
# y_test=y_test.reshape(-1,1)

In [ ]:
# y_pred

In [ ]:
# df=np.concatenate((y_test,y_pred),axis=1)

In [ ]:
# dataframe=pd.DataFrame(df,columns=['Rain on Tomorrow','Prediction of Rain'])

In [ ]:
# dataframe,df

####Calculating Accuracy


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier_logreg = LogisticRegression(solver='liblinear', random_state=0)


In [ ]:
classifier_logreg.fit(x_train, y_train)

In [ ]:
y_pred_LR = classifier_logreg.predict(x_test)
y_pred_LR

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy Score: {}".format(accuracy_score(y_test,y_pred_LR)))

In [ ]:
print("Train Data Score: {}".format(classifier_logreg.score(x_train, y_train)))
print("Test Data Score: {}".format(classifier_logreg.score(x_test, y_test)))

**Naive** **Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nbmodel = GaussianNB()


In [ ]:
nbmodel.fit(x_train,y_train)


In [ ]:
y_pred_nb = nbmodel.predict(x_test)

In [ ]:
y_pred_nb

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy Score: {}".format(accuracy_score(y_test,y_pred_nb)))

In [ ]:
print("Train Data Score: {}".format(nbmodel.score(x_train, y_train)))
print("Test Data Score: {}".format(nbmodel.score(x_test, y_test)))

Support Vector Machine

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm_clf = SVC(kernel='linear')

In [55]:
svm_clf.fit(x_train,y_train)

KeyboardInterrupt: ignored

In [ ]:
y_pred_svm = svm_clf.predict(x_test)

In [ ]:
y_pred_svm

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy Score: {}".format(accuracy_score(y_test,y_pred_svm)))

In [ ]:
print("Train Data Score: {}".format(svm_clf.score(x_train, y_train)))
print("Test Data Score: {}".format(svm_clf.score(x_test, y_test)))

**Artificial Neural Network**

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
ann_clf = tf.keras.models.Sequential()

In [ ]:
ann_clf.add(tf.keras.layers.Dense(units=6,activation="relu"))

In [ ]:
ann_clf.add(tf.keras.layers.Dense(units=6,activation="relu"))

In [ ]:
ann_clf.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

In [ ]:
ann_clf.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
ann_clf.fit(x_train,y_train,batch_size=32,epochs = 100)

In [ ]:
y_pred_ann = ann_clf.predict(x_test)

In [ ]:
y_pred_ann


In [ ]:
y_pred_ann = (y_pred_ann > 0.5)



In [ ]:
y_pred_ann

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy Score: {}".format(accuracy_score(y_test,y_pred_ann)))